In [6]:
# Import the modules
import pandas as pd
from pathlib import Path
#import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import sklearn as skl
import tensorflow as tf

In [13]:
# Read in the CSV file as a Pandas DataFrame
holiday_songs_spotify_df = pd.read_csv(
    Path("Holiday_Songs_Spotify.csv")
)

#C:\Users\wsabr\OneDrive\Desktop\project-4\data\Holiday_Songs_Spotify.csv
# Review the DataFrame
holiday_songs_spotify_df.head()

holiday_songs_spotify_df = holiday_songs_spotify_df.assign(Danceable = holiday_songs_spotify_df.danceability > 0.8)
holiday_songs_spotify_df

,Unnamed: 0,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,duration_ms,time_signature,key_mode,playlist_name,playlist_img,track_name,artist_name,album_name,album_img,Danceable
0,1,00IqwkT0PZhJ86PJajRCqk,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,...,213107,3,A# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Silver Bells,Johnny Mathis,Merry Christmas,https://i.scdn.co/image/b878b9e27201163be07e74...,False
1,2,01h424WG38dgY34vkI3Yd0,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,...,183613,4,A major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,White Christmas,Bing Crosby,White Christmas,https://i.scdn.co/image/3bb0daf5f87a737ce67ace...,False
2,3,08BhfyKUXxZrnyHrDavNHP,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,...,199093,3,F# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,The Christmas Waltz,Tony Bennett,A Swingin' Christmas Featuring The Count Basie...,https://i.scdn.co/image/96aa4fb09e7fe9d38599c8...,False
3,4,095XSaT8I2uI6Uldj2QrSl,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,...,178680,4,A major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Stop the Cavalry,Jona Lewie,On The Other Hand There's A Fist (Remastered),https://i.scdn.co/image/50c0ea35cacbf7c5d495c7...,False
4,5,09b2gJR45Pyip2rx9CnXW1,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,...,203404,4,F# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,I Don't Want to Go Home for Christmas,Independent Counsel of Funk,I Don't Want to Go Home for Christmas,https://i.scdn.co/image/77eb7c17cafe55037a1ab2...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,163,7qtBirjfexEIOK3xuqYLR3,0.572,0.895,F,-2.654,major,0.0644,0.013400,0.000000,...,147987,4,F major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Run Run Rudolph,Kelly Clarkson,Wrapped In Red,https://i.scdn.co/image/df6a48dc94245be056b352...,False
163,164,7rrVN7f83kwaz2sGxoFnJY,0.570,0.292,F,-14.408,major,0.0293,0.841000,0.000000,...,207000,4,F major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Christmas in Our Hearts,Jose Mari Chan,Christmas in Our Hearts,https://i.scdn.co/image/349276cbdfd8857b3975a8...,False
164,165,7t5i1MPAiJZcVRpzQtzzo6,0.838,0.346,G,-8.406,major,0.0471,0.827000,0.011100,...,155347,4,G major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,I Want a Hippopotamus for Christmas (Hippo the...,Gayla Peevey,Christmas Hits,https://i.scdn.co/image/939adaa29092810cc1c449...,True
165,166,7uVnnj80WNEu02JQ3BA3HJ,0.350,0.377,D#,-11.658,major,0.0340,0.353000,0.000000,...,222760,4,D# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,When a Child Is Born,Johnny Mathis,Christmas Hits,https://i.scdn.co/image/939adaa29092810cc1c449...,False


# New Section

In [14]:
#drop unneeded columns
holiday_songs_features_df = holiday_songs_spotify_df.drop(['Unnamed: 0','key_mode','time_signature', 'playlist_name', 'playlist_img',
                                                           'track_name', 'artist_name','album_name', 'album_img', 'track_uri'],axis=1)
holiday_songs_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,Danceable
0,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,0.1260,0.262,166.824,213107,False
1,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,0.4040,0.190,96.013,183613,False
2,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,0.0987,0.356,108.043,199093,False
3,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,0.0613,0.746,97.575,178680,False
4,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,0.2570,0.571,119.954,203404,False


In [15]:
regression_songs_df = holiday_songs_features_df.copy()


Linear Regression Model

In [16]:
regression_songs_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,Danceable
0,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,0.1260,0.262,166.824,213107,False
1,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,0.4040,0.190,96.013,183613,False
2,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,0.0987,0.356,108.043,199093,False
3,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,0.0613,0.746,97.575,178680,False
4,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,0.2570,0.571,119.954,203404,False


In [17]:
songs_final_df = regression_songs_df.drop(['Danceable', 'duration_ms'], axis=1)
songs_final_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,0.1260,0.262,166.824
1,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,0.4040,0.190,96.013
2,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,0.0987,0.356,108.043
3,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,0.0613,0.746,97.575
4,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,0.2570,0.571,119.954


In [18]:
#remove categories and replace with dummies
dummies_key = songs_final_df['key'].astype(str).str.get_dummies()
print(dummies_key)
dummies_mode = songs_final_df['mode'].astype(str).str.get_dummies()
print(dummies_mode)

     A  A#  B  C  C#  D  D#  E  F  F#  G  G#
0    0   1  0  0   0  0   0  0  0   0  0   0
1    1   0  0  0   0  0   0  0  0   0  0   0
2    0   0  0  0   0  0   0  0  0   1  0   0
3    1   0  0  0   0  0   0  0  0   0  0   0
4    0   0  0  0   0  0   0  0  0   1  0   0
..  ..  .. .. ..  .. ..  .. .. ..  .. ..  ..
162  0   0  0  0   0  0   0  0  1   0  0   0
163  0   0  0  0   0  0   0  0  1   0  0   0
164  0   0  0  0   0  0   0  0  0   0  1   0
165  0   0  0  0   0  0   1  0  0   0  0   0
166  0   0  0  0   0  1   0  0  0   0  0   0

[167 rows x 12 columns]
     major  minor
0        1      0
1        1      0
2        1      0
3        1      0
4        1      0
..     ...    ...
162      1      0
163      1      0
164      1      0
165      1      0
166      1      0

[167 rows x 2 columns]


In [19]:
songs_final_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,0.1260,0.262,166.824
1,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,0.4040,0.190,96.013
2,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,0.0987,0.356,108.043
3,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,0.0613,0.746,97.575
4,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,0.2570,0.571,119.954


In [20]:
#merge dummy data into DF
data = [songs_final_df, dummies_key, dummies_mode]
songs_final_df = pd.concat(data, axis=1)
songs_final_df = songs_final_df.drop(['key', 'mode'], axis=1)

In [21]:
songs_final_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,A,...,C#,D,D#,E,F,F#,G,G#,major,minor
0,0.195,0.348,-10.106,0.0332,0.820000,0.000000,0.1260,0.262,166.824,0,...,0,0,0,0,0,0,0,0,1,0
1,0.225,0.248,-15.871,0.0337,0.912000,0.000143,0.4040,0.190,96.013,1,...,0,0,0,0,0,0,0,0,1,0
2,0.444,0.288,-11.793,0.0326,0.911000,0.000007,0.0987,0.356,108.043,0,...,0,0,0,0,0,1,0,0,1,0
3,0.687,0.496,-4.708,0.0339,0.434000,0.001440,0.0613,0.746,97.575,1,...,0,0,0,0,0,0,0,0,1,0
4,0.477,0.841,-5.172,0.0358,0.000165,0.097100,0.2570,0.571,119.954,0,...,0,0,0,0,0,1,0,0,1,0


In [22]:
y = songs_final_df['danceability']

In [23]:
#drop unneeded columns
#X = songs_final_df.drop(['energy', 'A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#' ], axis=1)
X= songs_final_df.drop(columns='danceability')

In [24]:
y[:5]

0    0.195
1    0.225
2    0.444
3    0.687
4    0.477
Name: danceability, dtype: float64

In [25]:
X

,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,A,A#,...,C#,D,D#,E,F,F#,G,G#,major,minor
0,0.348,-10.106,0.0332,0.820000,0.000000,0.1260,0.262,166.824,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0.248,-15.871,0.0337,0.912000,0.000143,0.4040,0.190,96.013,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0.288,-11.793,0.0326,0.911000,0.000007,0.0987,0.356,108.043,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0.496,-4.708,0.0339,0.434000,0.001440,0.0613,0.746,97.575,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0.841,-5.172,0.0358,0.000165,0.097100,0.2570,0.571,119.954,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0.895,-2.654,0.0644,0.013400,0.000000,0.2160,0.922,145.008,0,0,...,0,0,0,0,1,0,0,0,1,0
163,0.292,-14.408,0.0293,0.841000,0.000000,0.1370,0.399,72.534,0,0,...,0,0,0,0,1,0,0,0,1,0
164,0.346,-8.406,0.0471,0.827000,0.011100,0.2130,0.930,113.988,0,0,...,0,0,0,0,0,0,1,0,1,0
165,0.377,-11.658,0.0340,0.353000,0.000000,0.1900,0.277,124.559,0,0,...,0,0,1,0,0,0,0,0,1,0


In [26]:
# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [27]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

In [29]:
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=X_train_scaled.shape[1]))

In [30]:
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))

In [31]:
# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))

In [32]:
# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                345       
                                                                 
 dense_1 (Dense)             (None, 5)                 80        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 431 (1.68 KB)
Trainable params: 431 (1.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
X_train_scaled.shape[1]

22

In [34]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])

# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 0.2872 - mae: 0.5130
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 0.2808 - mae: 0.5067
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2743 - mae: 0.4998
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2663 - mae: 0.4914
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2584 - mae: 0.4827
Epoch 6/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2473 - mae: 0.4701
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2337 - mae: 0.4539
Epoch 8/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2207 - mae: 0.4368
Epoch 9/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2056 - mae: 0.4182
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1899 - mae: 0.3970
Epoch 11/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1740 

In [35]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.0163 - mae: 0.1032 - 461ms/epoch - 230ms/step
Loss: 0.016252627596259117, Accuracy: 0.10318318009376526


In [45]:
# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [46]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

In [48]:
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=35, activation="relu", input_dim=X_train_scaled.shape[1]))

In [49]:
# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))

In [50]:
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))

In [51]:
# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 35)                805       
                                                                 
 dense_7 (Dense)             (None, 25)                900       
                                                                 
 dense_8 (Dense)             (None, 1)                 26        
                                                                 
Total params: 1731 (6.76 KB)
Trainable params: 1731 (6.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
# Compile the Sequential model together and customize metrics - optimized
nn_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])

# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 4ms/step - loss: 0.2915 - mae: 0.5165
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2856 - mae: 0.5099
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2780 - mae: 0.5012
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2707 - mae: 0.4928
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2576 - mae: 0.4772
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2408 - mae: 0.4537
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2226 - mae: 0.4213
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 0.1961 - mae: 0.3875
Epoch 9/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1761 - mae: 0.3635
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1667 - mae: 0.3497
Epoch 11/100
4/4 [==============================] - 0s 4ms/step - loss: 0.1495 

In [53]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.0452 - mae: 0.1570 - 109ms/epoch - 54ms/step
Loss: 0.045237667858600616, Accuracy: 0.15700232982635498


In [ ]:
from sklearn.metrics import mean_absolute_error, accuracy_score
# Example for regression
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')


Mean Absolute Error: 0.13156547619047618


In [ ]:
 # Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler

# Display all of the columns
pd.set_option('display.max_columns', None)

In [ ]:
# Instantiate the model with k = 3 neighbors
model = KNeighborsRegressor(n_neighbors=4)

In [ ]:
# Train the model
model.fit(X_train_scaled, y_train)

KNeighborsRegressor(n_neighbors=4)

In [ ]:
KNeighborsRegressor(n_neighbors=4)

KNeighborsRegressor(n_neighbors=4)

In [ ]:
# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array([0.5395 , 0.666  , 0.59325, 0.53525, 0.51575, 0.314  , 0.53525,
       0.53375, 0.5865 , 0.54475, 0.35325, 0.49975, 0.3045 , 0.4535 ,
       0.533  , 0.5055 , 0.62425, 0.6    , 0.52775, 0.4995 , 0.49075,
       0.6315 , 0.40425, 0.539  , 0.3045 , 0.49975, 0.60625, 0.539  ,
       0.54475, 0.56275, 0.39975, 0.47   , 0.48675, 0.52675, 0.44625,
       0.52775, 0.40425, 0.38275, 0.377  , 0.327  , 0.5015 , 0.654  ])

In [ ]:
X


,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,A,A#,B,C,C#,D,D#,E,F,F#,G,G#,major,minor
0,0.348,-10.106,0.0332,0.820000,0.000000,0.1260,0.262,166.824,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,0.248,-15.871,0.0337,0.912000,0.000143,0.4040,0.190,96.013,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0.288,-11.793,0.0326,0.911000,0.000007,0.0987,0.356,108.043,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,0.496,-4.708,0.0339,0.434000,0.001440,0.0613,0.746,97.575,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0.841,-5.172,0.0358,0.000165,0.097100,0.2570,0.571,119.954,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0.895,-2.654,0.0644,0.013400,0.000000,0.2160,0.922,145.008,0,0,0,0,0,0,0,0,1,0,0,0,1,0
163,0.292,-14.408,0.0293,0.841000,0.000000,0.1370,0.399,72.534,0,0,0,0,0,0,0,0,1,0,0,0,1,0
164,0.346,-8.406,0.0471,0.827000,0.011100,0.2130,0.930,113.988,0,0,0,0,0,0,0,0,0,0,1,0,1,0
165,0.377,-11.658,0.0340,0.353000,0.000000,0.1900,0.277,124.559,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Combine relevant columns into a single column
holiday_songs_spotify_df["combined_features"] = holiday_songs_spotify_df["track_name"] + '|' + holiday_songs_spotify_df["artist_name"] + '|' + holiday_songs_spotify_df["album_name"]

In [ ]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

NameError: ignored

In [ ]:
# Fit and transform the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

In [ ]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
import random
features =['energy_input', 'key_input', 'loudness_input', 'mode_input', 'speechiness_input', 'instrumentalness_input', 'liveness_input', 'acousticness_input']

def get_recommendations(features, y_pred, cosine_sim=cosine_sim):
    index = df[df["track_name"].str.lower() == features].index[0]

    # danceability_mapping = {
    #     'low': 0.2,
    #     'medium': 0.5,
    #     'high': 0.8
    # }

    # danceability_threshold = danceability_mapping.get(danceability_category.lower())

    # if danceability_threshold is None:
    #     return "Invalid danceability category. Please choose 'low', 'medium', or 'high'."

    # # Filter songs based on danceability threshold
    # filtered_df = df[(df['danceability'] >= danceability_threshold) & (df.index != index)]

    # # If no songs meet the threshold, return a message
    # if filtered_df.empty:
    #     return f"No songs found with {danceability_category} danceability."

    # Shuffle the indices for randomness
    shuffled_indices = list(filtered_df.index)
    random.shuffle(shuffled_indices)

    # Get recommendations based on the shuffled indices
    recommendations = []
    for shuffled_index in shuffled_indices:
        sim_scores = list(enumerate(cosine_sim[shuffled_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[:4]  # Get top 4 recommendations
        song_indices = [i[0] for i in sim_scores]
        recommendations.extend(df[["track_name", "artist_name", "album_name"]].iloc[song_indices].values)

    return recommendations[:4]  # Limit the final recommendations to 4


NameError: ignored

In [ ]:
# Main loop to get user input and provide recommendations
while True:
    print("Type 'exit' to quit")
    energy_input = input("Merry Christmas! Enter the desired energy level ('low', 'medium', 'high'): ").lower()
    key_input = input("Enter the desired key ('A', 'A#', 'B','C#', 'D', 'D#', 'F', 'F#', 'G', 'G#'): ").lower()
    loudness_input = input("Enter the desired loudness ('soft', 'medium', 'loud'): ").lower()
    mode_input = input("Enter the desired mode ('major', 'minor'): ").lower()
    speechiness_input = input("Enter the desired speechiness('speechy', 'not speechy'): ").lower()
    instrumentalness_input = input("Enter the desired instrumentalness ('instrumental', 'not instrumental'): ").lower()
    liveness_input = input("Enter desired livenss level ('live audience', 'no audience'): ").lower()
    acousticness_input = ("Enter the desired acousticness ('acoustic', 'not acoustic'): ").lower()

    if features == 'exit':
        break

    # if song_input not in df["track_name"].values:
    #     print(f"Song '{song_input}' not found in the dataset. Please check the spelling or enter a different Christmas song name.")
    #     continue

    # danceability_input = input("Enter danceability category ('low', 'medium', 'high'): ")

    # recommendations = get_recommendations(y_pred, features)
    # print(f"Recommended songs for '{features}' with {y_pred} danceability:")

    # if recommendations:
    #     for recommendation in recommendations:
    #         print(recommendation)
    # else:
    #     print("No recommendations found.")

Type 'exit' to quit
Type 'exit' to quit
